In [7]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import math
import cv2
from PIL import Image
from scipy import misc
from os import listdir
from os.path import isfile, join
from collections import Counter
import matplotlib.pyplot as plt
import tensorflow as tf

In [8]:
from keras.models import Sequential,Input,Model
from keras.layers import Conv2D
from keras.layers import MaxPooling2D,AveragePooling2D
from keras.layers import Flatten,LeakyReLU
from keras.layers import Dense,Dropout,BatchNormalization

In [9]:
classifier = Sequential()

# Step 1 - Convolution
classifier.add(Conv2D(filters=64, kernel_size=2, padding='same', activation='relu', input_shape=(64,64,3)))
classifier.add(BatchNormalization())

# Step 2 - Pooling
classifier.add(MaxPooling2D(pool_size = (2, 2)))
classifier.add(BatchNormalization())

# Adding a second convolutional layer
classifier.add(Conv2D(filters=32,kernel_size=2,padding='same',activation='tanh'))
classifier.add(MaxPooling2D(pool_size =(2,2)))
classifier.add(BatchNormalization())


# Adding a third convolutional layer
classifier.add(Conv2D(filters=16,kernel_size=2,padding='same',activation='tanh'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))
classifier.add(BatchNormalization())

# Step 3 - Flattening
classifier.add(Flatten())

# Step 4 - Full connection
classifier.add(Dense(units=32,activation = 'tanh'))
classifier.add(Dense(units=64,activation = 'tanh'))
classifier.add(Dropout(0.5))
classifier.add(Dense(units=128,activation = 'tanh'))
classifier.add(Dropout(0.5))
classifier.add(Dense(units=256,activation = 'tanh'))
classifier.add(Dropout(0.5))
classifier.add(Dense(units = 8,activation = 'softmax'))

In [10]:
#model = Model(inputs=inputs, outputs=[age_model,gender_model])
#classifier.compile(optimizer = 'adam', loss =['categorical_crossentropy'],metrics=['accuracy'])
classifier.compile(optimizer='rmsprop', loss=['categorical_crossentropy'], metrics=['accuracy'])

In [11]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
test_datagen =  ImageDataGenerator(rescale = 1./255)

print("\nTraining the data...\n")

training_set = train_datagen.flow_from_directory(r'C:\Users\sathe\Satheesh_SK\No_Need\Sample_Data\Training',
                                                 target_size = (64, 64),
                                                 batch_size = 12,                                       
                                                 class_mode = 'categorical')


test_set = test_datagen.flow_from_directory(r'C:\Users\sathe\Satheesh_SK\No_Need\Sample_Data\Testing',
                                            target_size = (64, 64),
                                            batch_size=12,
                                            class_mode = 'categorical')


Training the data...

Found 18896 images belonging to 8 classes.
Found 2702 images belonging to 8 classes.


In [12]:
model = classifier.fit_generator(training_set,
                                 steps_per_epoch=600,
                                 epochs = 20,
                                 validation_data = test_set,    
                                 validation_steps = 50)

Epoch 1/20
600/600 [==============================] - 66s 108ms/step - loss: 1.9846 - accuracy: 0.3012 - val_loss: 1.8833 - val_accuracy: 0.3267
Epoch 2/20
600/600 [==============================] - 63s 104ms/step - loss: 1.5955 - accuracy: 0.4373 - val_loss: 1.6704 - val_accuracy: 0.4033
Epoch 3/20
600/600 [==============================] - 64s 106ms/step - loss: 1.4226 - accuracy: 0.4852 - val_loss: 1.8251 - val_accuracy: 0.3667
Epoch 4/20
600/600 [==============================] - 64s 106ms/step - loss: 1.3576 - accuracy: 0.5003 - val_loss: 1.7987 - val_accuracy: 0.4250
Epoch 5/20
600/600 [==============================] - 68s 114ms/step - loss: 1.2767 - accuracy: 0.5433 - val_loss: 1.3012 - val_accuracy: 0.5217
Epoch 6/20
600/600 [==============================] - 70s 114ms/step - loss: 1.2232 - accuracy: 0.5556 - val_loss: 1.6153 - val_accuracy: 0.4133
Epoch 7/20
600/600 [==============================] - 70s 115ms/step - loss: 1.2136 - accuracy: 0.5549 - val_loss: 1.2522 - val_ac

In [13]:
model=classifier.save('multiclass_model1.h5')

In [14]:
train_samples = len(training_set.filenames)
train_labels = training_set.classes 
num_classes = len(training_set.class_indices)
train_samples,num_classes,train_labels,len(train_labels)

(18896, 8, array([0, 0, 0, ..., 7, 7, 7]), 18896)

In [15]:
test_samples = len(test_set.filenames)
test_labels = test_set.classes 
num_classes = len(test_set.class_indices)
test_samples,num_classes,test_labels

(2702, 8, array([0, 0, 0, ..., 7, 7, 7]))

In [16]:
(eval_loss, eval_accuracy) = classifier.evaluate(test_set, batch_size=50,verbose=1)

226/226 [==============================] - 5s 18ms/step - loss: 1.3253 - accuracy: 0.4778


In [17]:
preds=classifier.predict(test_set,test_labels)
print('Predictions:',np.round(preds)[1])

Predictions: [0. 0. 1. 0. 0. 0. 0. 0.]


# New Prediction for Face and Age:

In [18]:
from keras.preprocessing import image
from PIL import Image
from keras.models import load_model
age_model = load_model('age_model.h5')

In [19]:
import imutils
path=r'C:\Users\sathe\Satheesh_SK\No_Need\Sample_Data\Testing\MiddleAgeMen\26_0_2_20170116175516176.jpg.chip.jpg'
image = cv2.imread(path)
output = imutils.resize(image, width=400)

# pre-process the image for classification
image = cv2.resize(image, (64, 64))
image = image.astype("float") / 255.0
image = tf.keras.preprocessing.image.img_to_array(image)
image = np.expand_dims(image, axis=0)

In [20]:
#mlb = pickle.loads(open(args["labelbin"], "rb").read())
proba = classifier.predict(image)[0]
idxs = np.argsort(proba,axis=0)
idxs

array([4, 5, 1, 0, 2, 6, 3, 7], dtype=int64)

In [38]:
proba

array([0.01824107, 0.00830903, 0.04619066, 0.08915819, 0.00189908,
       0.00521638, 0.05110139, 0.77988416], dtype=float32)

In [21]:
age_proba=age_model.predict(image)[0].astype(int)
age_proba

array([40])

In [25]:
labels=training_set.class_indices
labels

{'FemaleChild': 0,
 'MaleChild': 1,
 'MiddleAgeMen': 2,
 'MiddleAgeWomen': 3,
 'OldAgeMen': 4,
 'OldWomen': 5,
 'YoungMen': 6,
 'YoungWomen': 7}

In [26]:
inv_map = {v:k for k, v in labels.items()} 
inv_map

{0: 'FemaleChild',
 1: 'MaleChild',
 2: 'MiddleAgeMen',
 3: 'MiddleAgeWomen',
 4: 'OldAgeMen',
 5: 'OldWomen',
 6: 'YoungMen',
 7: 'YoungWomen'}

In [28]:
class_predicted = classifier.predict_classes(image)
print("ID: {}, Predicted Gender: {} Age :{} ".format(class_predicted[0],  inv_map[class_predicted[0]],age_proba[0]))

C:\Users\sathe\anaconda3\lib\site-packages\tensorflow\python\keras\engine\sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


ID: 7, Predicted Gender: YoungWomen Age :40 


In [37]:
(eval_loss, eval_accuracy) = classifier.evaluate(test_set, batch_size=50,verbose=1)

226/226 [==============================] - 24s 86ms/step - loss: 1.3253 - accuracy: 0.4778
